In [1]:
!pip install selenium
!pip install bs4

In [2]:
import selenium
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
weblink = "https://www.spotrac.com/mlb/rankings/2020/salary/"
response = requests.get(weblink)
read_text = response.text

soup = BeautifulSoup(read_text)

#Gathers all of the hyperlinks from the websites  (href = True) and prints the titles of them through the .text
for x in soup.find_all('a', href = True):
    print(x.text.strip())




PREMIUM
Sign In
Register

NFL
Team Salary Caps
Positional Spending
Contracts
Salary Rankings
Free Agents

Trackers & Tools
» Market Values
» Fines/Suspensions
» IR Tracker
» Depth Charts Tracker
» Draft Tracker
» Options
» College Tracker
Best Values
Transactions

NBA
Team Cap Tracker
Positional Payrolls
Contracts
Salary Rankings
Free Agents

Trackers & Tools
» Depth Charts
» Options
» Fines/Suspensions
» Draft Tracker
» College Tracker
» Awards Tracker
Best Values

Transactions
» Recent Transactions
» Extensions
» Trades

MLB
Team Payrolls
Luxury Tax Tracker
Positional Payrolls
Contracts
Salary Rankings
Free Agents
Prospects

Trackers & Tools
» Fines/Suspensions
» Injured List Tracker
» Options Tracker
» Trade Tracker
» Arbitration Tracker
Best Values
Transactions

NHL
Team Salary Caps
Positional Payrolls
Contracts
Salary Rankings
Free Agents

Trackers & Tools
» IR Tracker
» Options
» Fines/Suspensions
» Draft Tracker
» Trade Tracker
» College/Junior Team Tracker
» Country Tracker


# Beautiful Soup Implementation

In [9]:
response = requests.post(weblink, data={'ajax': 'true','mobile': 'false'}).content
read_text = response

soup = BeautifulSoup(read_text)
column_headers = soup.find_all('tr')[0]
column_headers = [i.text.strip().title() for i in column_headers.find_all('th')]
column_headers = column_headers[1:]

baseballPlayers = soup.find_all('tr')[1:]

playerName = []
position = []
age = []
salary = []
team = []

for player in baseballPlayers:
    playerName.append(player.h3.a.text)
    position.append(player.find('td', class_ = 'center small').text.strip())
    salary.append(player.find('span', class_ = 'info').text.strip().replace('$',""))
    team.append(player.find('div', class_ = 'rank-position').text.strip())
    for ages in player.find_all("td", {"class": "center small"}):
        plainText = ages.text.strip()
        
        if(plainText.isnumeric()):
            age.append(plainText)

playerData = pd.DataFrame({"Player": playerName, 'Team': team, 'Age':age,"Position" : position, "Salary": salary})
playerData

,Player,Team,Age,Position,Salary
0,Mike Trout,LAA,28,CF,"37,766,667"
1,Gerrit Cole,NYY,29,SP,"36,000,000"
2,Max Scherzer,WSH,35,SP,"35,920,616"
3,Nolan Arenado,COL,29,3B,"35,025,000"
4,Stephen Strasburg,WSH,31,SP,"35,000,000"
...,...,...,...,...,...
995,Riley Smith,ARI,25,RP,"563,500"
996,Keegan Akin,BAL,25,SP,"563,500"
997,Isaac Paredes,DET,21,INF,"563,500"
998,Pavin Smith,ARI,24,1B,"563,500"


# Pandas Implementation

In [5]:
#Downside only accesses the first 100 rows
#Since the rest of the data must be accesses by scrolling, if you wanted to use pandas read_html function you would
#have to use selenium to figure the rest out
dfs = pd.read_html(weblink, header = 0)[0].reset_index()
dfs = dfs[['Player','POS','Age','salary']]
dfs.columns = dfs.columns.str.title()
dfs.dropna(inplace = True)
dfs['Team'] = dfs['Player'].str.split(" ", n = 2, expand = True)[2]
dfs['Player'] = dfs['Player'].str[:-4]
dfs

,Player,Pos,Age,Salary,Team
0,Mike Trout,CF,28,"$37,766,667",LAA
1,Gerrit Cole,SP,29,"$36,000,000",NYY
2,Max Scherzer,SP,35,"$35,920,616",WSH
3,Nolan Arenado,3B,29,"$35,025,000",COL
4,Stephen Strasburg,SP,31,"$35,000,000",WSH
...,...,...,...,...,...
95,Will Smith,RP,30,"$13,000,000",ATL
96,Josh Reddick,RF,33,"$13,000,000",HOU
97,Randal Grichuk,CF,28,"$13,000,000",TOR
98,Jose Abreu,1B,33,"$12,781,666",CHW


In [7]:
#Want to scrape 25 pages from the 2020 season to compare players and salary
hitData = pd.DataFrame()
link = "https://www.mlb.com/stats/doubles/2020"
for i in range(2, 25):
    r = requests.get(link)
    hitData = pd.concat([hitData, (pd.read_html(r.text, header = 0)[0])])
    link = f"https://www.mlb.com/stats/doubles/2020?page={i}"

hitData = hitData.reset_index()
hitData.drop('index', axis = 1, inplace = True)
hitData

,PLAYERPLAYER,TEAMTEAM,GG,ABAB,RR,HH,caret-upcaret-down2Bcaret-upcaret-down2B,3B3B,HRHR,RBIRBI,BBBB,SOSO,SBSB,CSCS,AVGAVG,OBPOBP,SLGSLG,OPSOPS
0,1FreddieF FreemanFreeman1B1‌‌‌,ATL,60,214,51,73,23,1,13,53,45,37,2,0,0.341,0.462,0.640,1.102
1,2DominicDo SmithSmith1B2‌‌‌,NYM,50,177,27,56,21,1,10,42,14,45,0,0,0.316,0.377,0.616,0.993
2,3CesarC HernándezHernandez2B3‌‌‌,CLE,58,233,35,66,20,0,3,20,24,57,0,0,0.283,0.355,0.408,0.763
3,4ChristianC WalkerWalker1B4‌‌‌,AZ,57,218,35,59,18,1,7,34,19,50,1,1,0.271,0.333,0.459,0.792
4,5JoseJ IglesiasIglesiasSS5‌‌‌,BAL,39,142,16,53,17,0,3,24,3,17,0,0,0.373,0.400,0.556,0.956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570,486BrianB NavarretoNavarretoC486‌‌‌,MIA,2,5,0,2,0,0,0,0,0,1,0,0,0.400,0.400,0.400,0.800
571,486HectorH NerisNerisP486‌‌‌,PHI,24,0,0,0,0,0,0,0,0,0,0,0,0.000,0.000,0.000,0.000
572,486JohnJ NogowskiNogowski1B486‌‌‌,STL,1,4,0,1,0,0,0,0,0,1,0,0,0.250,0.250,0.250,0.500
573,486AaronA NolaNolaP486‌‌‌,PHI,12,0,0,0,0,0,0,0,0,0,0,0,0.000,0.000,0.000,0.000


In [8]:
column_headers = ['Player', 'Team', 'Games Played', 'At Bats', 'Runs', 'Hits', 'Doubles', 'Triples', 'Home Runs', 'RBIs',
                 'Walks','Strikeouts', 'Stolen Bases', 'Caught Stealing', 'Batting Average', 'On Base Percentage', 
                 'Slugging Percentage', 'On Base Plus Slugging']

hitData.columns = column_headers

replaceNames = hitData['Player'].str.findall('[A-Z][^A-Z]*')

# df['Player'] = replaceNames
playerNames = []

for names in replaceNames:
    playerNames.append(names[0] + " " + names[2])
    
hitData['Player'] = playerNames
hitData

,Player,Team,Games Played,At Bats,Runs,Hits,Doubles,Triples,Home Runs,RBIs,Walks,Strikeouts,Stolen Bases,Caught Stealing,Batting Average,On Base Percentage,Slugging Percentage,On Base Plus Slugging
0,Freddie Freeman,ATL,60,214,51,73,23,1,13,53,45,37,2,0,0.341,0.462,0.640,1.102
1,Dominic Smith,NYM,50,177,27,56,21,1,10,42,14,45,0,0,0.316,0.377,0.616,0.993
2,Cesar Hernández,CLE,58,233,35,66,20,0,3,20,24,57,0,0,0.283,0.355,0.408,0.763
3,Christian Walker,AZ,57,218,35,59,18,1,7,34,19,50,1,1,0.271,0.333,0.459,0.792
4,Jose Iglesias,BAL,39,142,16,53,17,0,3,24,3,17,0,0,0.373,0.400,0.556,0.956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570,Brian Navarreto,MIA,2,5,0,2,0,0,0,0,0,1,0,0,0.400,0.400,0.400,0.800
571,Hector Neris,PHI,24,0,0,0,0,0,0,0,0,0,0,0,0.000,0.000,0.000,0.000
572,John Nogowski,STL,1,4,0,1,0,0,0,0,0,1,0,0,0.250,0.250,0.250,0.500
573,Aaron Nola,PHI,12,0,0,0,0,0,0,0,0,0,0,0,0.000,0.000,0.000,0.000


In [17]:
#Inner join
overallData = pd.merge(playerData, hitData, on = ['Player','Team'], how = 'inner')
overallData['Salary'] = overallData['Salary'].str.replace(',', "").astype(float)
overallData.dtypes

Player                    object
Team                      object
Age                       object
Position                  object
Salary                   float64
Games Played               int64
At Bats                    int64
Runs                       int64
Hits                       int64
Doubles                    int64
Triples                    int64
Home Runs                  int64
RBIs                       int64
Walks                      int64
Strikeouts                 int64
Stolen Bases               int64
Caught Stealing            int64
Batting Average          float64
On Base Percentage       float64
Slugging Percentage      float64
On Base Plus Slugging    float64
dtype: object